# Exporing the NASA Exoplanet Archive Tables 

The NASA Exoplanet Archive thoughtfully and painstakingly describes [the data columns](https://exoplanetarchive.ipac.caltech.edu/docs/API_PS_columns.html) it contains, as well as [how they merge the bazillions of different measurements from different references](https://exoplanetarchive.ipac.caltech.edu/docs/pscp_calc.html). 

*Here, we explore some visualizations of how the Planetary Systems (one row per reference) and Planetary Systems Composite (one row per planet) tables compare to one another. This is a very crude overview, designed mostly to help with weird nitty gritty internal decision making for the `exoatlas`. We include these in the documentation because it's useful to see these plots generated regularly as the Exoplanet Archive grows, but we hide it away at the end because we don't explain things particularly well (yet?).*

In [ ]:
from exoatlas import * 
explorer = NASAExoplanetArchiveComparison()

## What's in which table? 

First, let's take a few parameters to see how the contents of the Planetary Systems Composite Parameters table to the complete contents of the Planetary Systems table from which it is derived. Entirely (and foolishly) negelecting uncertainties on values and which values are limits, let's make some plots indicating the complete set of values that appear in `ps` for each planet + parameter in `pscomppars`.

In [ ]:
explorer.do_completeness_check()

## 

## Are there weird unit problems? 

Weirdly, for some planets we'll get different answers whether we ingest the planet quantities like mass and radius in Earth units or in Jupiter units. This might be caused by units and/or rounding errors in the papers that got ingested into the archive. It's probably good to be aware of!

In [ ]:
explorer.do_units_check()

## Do masses and radii make sense? 

In the Planetary Systems Composite Parameters table, the archive calculates [mass and radius estimates for planets that don't have measurements from an empirical mass-radius relationship](https://exoplanetarchive.ipac.caltech.edu/docs/pscp_calc.html). An easy way to remove those theoretical estimates, is to eliminate values that don't have finite uncertainties associated with them. All measurements should have uncertainties! I believe this cut also removes some upper/lower limits (but don't quote me on that, yet).

In [ ]:
explorer.do_mass_radius_check()

## Are stellar luminosities, radii, and effective temperatures consistent?

## Are composite transit periods and midpoints safe to use? 